# CHALLENGE OPTION
### Eduardo Álvarez
  

<span style="color:red">"SISTEMA DE RECOMENDACIÓN ANIME"</span>

Tradicionalmente, en la [literatura](https://data-flair.training/blogs/data-science-r-movie-recommendation/) encontramos principalmente dos sistemas de recomendación (ver Fig. más abajo):

1- Los filtros colaborativos (collaborative filtering), que se basan en las características del usuario. El sistema analiza las compras anteriores, las preferencias, las calificaciones que ha dado de otros productos, el importe medio de las compras, etc. y busca otros usuarios que se parecen a él y que han tomado decisiones parecidas. Los productos que han tenido éxito con usuarios similares, seguramente también le interesarán al nuevo usuario.

2- Los filtros basados en contenido (content-based filtering), que utilizan las características del artículo (marca, precio, calificaciones, tamaño, categoría, etc.) para hacer las recomendaciones. 

![Image of Yaktocat](https://d2h0cx97tjks2p.cloudfront.net/blogs/wp-content/uploads/sites/2/2019/07/data-science-movie-recommendation-project.jpg)

# Enunciado desafío

El objetivo del desafío es desarrollar un sistema de recomendación que permita sugerir anime que los
usuarios no han visto, en base a sus ratings previos. Es muy importante justificar la elección del sistema
(o modelo), el trabajo previo de la data (EDA) y la documentación de lo que se hizo (no es necesario un
informe, pero si comentar porqué se tomaron las decisiones que se tomaron; por ej eliminar una variable
o eliminar registros missing, etc).

# Elección modelo

Se eligió el el sistema de recomendación basado "Collaborative Filtering (CF)".
  

Python, posee una librería para sistemas de recomendación llamada [Surprise](http://surpriselib.com/), fuertemente documentada, con varios algoritmos de predicción listos para usar. Además, esta librería proporciona herramientas para evaluar, analizar y comparar el rendimiento de los algoritmos, como validación cruzada.

![Image of Yaktocat](https://i.imgur.com/z5Rqeey.jpg)

En la documentación se publican los tiempos promedio de ejecución, RMSE, MAE y total de varios algoritmos (con sus parámetros predeterminados) en un procedimiento de validación cruzada de 5 veces. Los conjuntos de datos son los conjuntos de datos Movielens 100k y 1M.

![Image of Yaktocat](https://i.imgur.com/cRoalwM.jpg)

En esta tabla se aprecia que el error cuadrático medio es menor en Singular value descomposition (SVDpp). Cuanto menor sea el RMSE, mejor será el rendimiento. Sin embargo, este algoritmo es el que posee mayor Time.

Esencialmente, este algoritmo transforma el desafío de recomendación en un problema de optimización. 

[ver más](https://hackernoon.com/introduction-to-recommender-system-part-1-collaborative-filtering-singular-value-decomposition-44c9659c5e75/)

[Paper SVD](http://papers.nips.cc/paper/3208-probabilistic-matrix-factorization.pdf)

# Librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
pd.options.display.max_rows = 10
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import os

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

import surprise
from surprise import Dataset
from surprise import Reader
from surprise import SVDpp
from surprise import model_selection
from surprise.model_selection import cross_validate, GridSearchCV

import warnings
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
%matplotlib inline


# Análisis exploratorio de datos (AED)

In [ ]:
#user = pd.read_csv(r'C:\Users\NuuBI\Documents\deafio Option\rating.csv')
#anime = pd.read_csv(r'C:\Users\NuuBI\Documents\deafio Option\anime.csv')
df_rating = pd.read_csv('../input/rating.csv')
df_anime = pd.read_csv('../input/anime.csv')

# Data frame anime

In [ ]:
df_anime.shape


In [ ]:
df_anime.dtypes

Se eliminan duplicados del df_anime


In [ ]:
print(df_anime.shape)
print(df_anime.drop_duplicates().shape)
print(df_anime.info())
df_anime.tail(10)

In [ ]:
print(df_anime.isnull().sum())

Hay valores nulos en Genre, type y rating

### VARIABLE EPISODES

Variabe episodes, contiene una categoría "Unknown".
Se procede a cambiar "Unknown" por un valor distintivo "-1" (para no borrar información" y luego transformar el tipo de dato a numérico.

In [ ]:
df_anime.episodes.value_counts()

In [ ]:
df_anime.loc[df_anime.episodes == 'Unknown', 'episodes'] = -1
df_anime["episodes"] = df_anime["episodes"].astype(float)
df_anime.dtypes

In [ ]:
print(df_anime.isnull().sum())

In [ ]:
df_anime.episodes.describe()

In [ ]:
df_anime.episodes.hist()

### VARIABLE GENRE

Tenemos 62 filas con valores nulos en genero.
Se creará una categoría para los nulos llamada "nogenre"

In [ ]:
df_anime.loc[df_anime.genre.isnull()]

In [ ]:
df_anime["genre"].replace(np.nan, "nogenre", inplace=True)
print(df_anime.isnull().sum())

### VARIABLE TYPE

In [ ]:
df_anime.loc[df_anime.type.isna()]

Todos los valore nulos de Type coinciden con valores nulos de episodes y rating. Se procede a eliminar valores nulos de type.

In [ ]:
df_anime = df_anime.dropna(0, subset=['type'])
pd.isnull(df_anime).sum()

### VARIABLE RATING

In [ ]:
df_anime.loc[pd.isna(df_anime.rating)]

Tomé la decisión de eliminar valores nulos de rating. Lo más probable es que estos anime aún no hayan vistos, por lo que simplemente no podrían ser recomendamos.


In [ ]:
df_anime = df_anime.dropna(0, subset=['rating'])
pd.isnull(df_anime).sum()

In [ ]:
df_anime=df_anime.reset_index()

In [ ]:
df_anime['rating'].hist()

# Data frame rating

In [ ]:
df_rating.shape

In [ ]:
df_rating.count()

In [ ]:
df_rating.dtypes

In [ ]:
pd.isnull(df_rating).sum()

In [ ]:
df_rating.head()

In [ ]:
df_rating.rating.hist()

Si bien una gran parte de este dataframe se compone de valores -1(usuario vio pero no evaluó) estos estan distribuidos por usuario en pequeñas porciones. Tomando en cuenta que el objetivo es encontrar similitudes entre los gustos de los usuarios, se toma la decisión de eliminar los valores con -1.

In [ ]:
df_rating

In [ ]:
df_rating = df_rating[df_rating.rating != -1]
df_rating.shape

# Sistema de recomendación


In [ ]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
%matplotlib inline

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

import surprise
from surprise import Dataset
from surprise import Reader
from surprise import SVDpp
from surprise import model_selection
from surprise.model_selection import cross_validate, GridSearchCV

### Collaborative Filtering Recommendation System

In [ ]:
df_rating.head()

In [ ]:
data = df_rating['rating'].value_counts().sort_index(ascending=False)
trace = go.Bar(x = data.index,
               text = ['{:.1f} %'.format(val) for val in (data.values / df_rating.shape[0] * 100)],
               textposition = 'auto',
               textfont = dict(color = '#000000'),
               y = data.values,
               )

layout = dict(title = 'Distribution Of {} moive-ratings'.format(df_rating.shape[0]),
              xaxis = dict(title = 'Rating'),
              yaxis = dict(title = 'Count'))

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

La mayoría de los rating se encuentran cercanos a la nota 8.

Se procede a seleccionar una parte del de la base de datos (60.000) para mostrar el sistema de recomendación

In [ ]:
df = df_rating.iloc[:60000,].reset_index()
df = df.drop(['index'], axis=1)
df.head()

### Construyendo el modelo de recomendación

In [ ]:
df['rating'].unique()

In [ ]:
recmodel = SVDpp()
reader = Reader(rating_scale=(1,10))
df_rating_rec = Dataset.load_from_df(df, reader)
recmodel.fit(df_rating_rec.build_full_trainset()) 
cross_validate(recmodel, df_rating_rec, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [ ]:
anime_id = df['anime_id'].unique()
# Tomamos al usuario 1 como ejemplo
anime_id1 = df.loc[df['user_id'] == 1, 'anime_id']
anime_id_to_pred = np.setdiff1d(anime_id, anime_id1)

In [ ]:
#Testset
testset = [[1, anime_id, 10] for anime_id in anime_id_to_pred]
user_id1_pred = recmodel.test(testset)
df_pred = pd.DataFrame(user_id1_pred)
df_pred.head()

In [ ]:
df_pred = df_pred.rename(columns={'uid': 'user_id', 'iid': 'anime_id', 'est': 'predicted rating'})
df_pred = df_pred.drop(['r_ui', 'details'], axis=1)
df_pred.head()

In [ ]:
# Rating predichos
df_pred = df_pred.sort_values('predicted rating', ascending=False)
df_pred_anime_id = df_pred.head(10)['anime_id']

df_recommendation = pd.DataFrame({'anime_id':[], 'name':[], 'genre':[], 'type':[], 'episodes':[], 'rating':[], 'members':[]})

for anime_id in df_pred_anime_id:
    df_recommendation = df_recommendation.append(df_anime[df_anime['anime_id'] == anime_id])
        
df_recommendation = df_recommendation.reset_index()
df_recommendation = df_recommendation.drop(['index'], axis=1)
df_recommendation['anime_id'] = df_recommendation['anime_id'].astype('int')
df_recommendation

### Función recomendar

In [ ]:
#Creación función con un primer parametro para seleccionar el usuario y un segundo parámetro para el número de animes a recomendar. 
def obtener_recomendaciones_option(user_id, num_recommendations):
    anime_id = df['anime_id'].unique()
    anime_id_user = df.loc[df['user_id'] == user_id, 'anime_id']
    anime_id_to_pred = np.setdiff1d(anime_id, anime_id_user)
    testset = [[user_id, anime_id, 10] for anime_id in anime_id_to_pred]
    user_id_pred = recmodel.test(testset)
    df_pred = pd.DataFrame(user_id_pred)
    
    df_pred = df_pred.sort_values('est', ascending=False)
    df_pred_anime_id = df_pred.head(num_recommendations)['iid']

    df_recommendation = pd.DataFrame({'anime_id':[], 'name':[], 'genre':[], 'type':[], 'episodes':[], 'rating':[], 'members':[]})

    for anime_id in df_pred_anime_id:
        df_recommendation = df_recommendation.append(df_anime[df_anime['anime_id'] == anime_id])
        
    df_recommendation = df_recommendation.reset_index()
    df_recommendation = df_recommendation.drop(['index'], axis=1)
    df_recommendation['anime_id'] = df_recommendation['anime_id'].astype('int')
    return df_recommendation

In [ ]:
obtener_recomendaciones_option(1, 5)

## Para mejorar
Considerar desarrollar un modelo mixto basado en los usuarios y en los items. Es decir, considerar las variables, género, tipo y episodio.
